In [ ]:
'''Here we read in all necessary libraries'''
import numpy as np
import pandas as pd

from pylife.stress.rainflow import *
from pylife.strength.meanstress import *
import matplotlib.pyplot as plt
from pylife.stress.histogram import *
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import io
from IPython.display import HTML
import base64 
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

## Das ist mein 

## Import Data ##

In [ ]:
files = 'HPC-Load1-Year.xlsx'
data_loc = 'C:\\python\\RainFlowSOFC\\data\\'

In [ ]:
%matplotlib notebook
data_df = pd.read_excel(data_loc + files,sheet_name = 0).dropna()
display(data_df)
colName = "1-min"
# plt.plot(data_df["Timestamp"],data_df[colName])
plt.plot(data_df[colName])

## Rainflow Calcultion 

In [ ]:
rfc = RainflowCounterFKM().process(data_df[colName].values)
rfm = rfc.get_rainflow_matrix_frame(12)
display(rfm)

In [ ]:
counts = np.flipud((rfm.values.reshape(rfm.index.levshape).T)).ravel()

In [ ]:
%matplotlib notebook
colormap = cm.ScalarMappable()
# color = colormap.to_rgba(dz)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
bottom = np.zeros_like(counts)
froms = rfm.index.get_level_values('from').mid
tos = rfm.index.get_level_values('to').mid
width = depth = 150
cmap = cm.get_cmap('viridis')
max_height = np.max(counts)   # get range of colorbars
min_height = np.min(counts)
rgba = [cmap((k-min_height)/max_height) for k in counts] 
ax.bar3d(froms.ravel(), tos.ravel(), bottom, width, depth, counts, shade=True, color=rgba, zsort='average')


In [ ]:
%matplotlib notebook
amplitude = MeanstressHist(rfm)
amplitude = pd.DataFrame(data = counts, index = amplitude._Sa.ravel())
# display(amplitude)
range_only = rfm.meanstress_hist.FKM_goodman(pd.Series({'M': 0,'M2': 0}), R_goal = 0)
amplitude = range_only.index.get_level_values('range').mid.values[::-1]
cycles = range_only.values[::-1].ravel()
plt.barh(amplitude,np.cumsum(cycles), height = range_only.index.get_level_values('range').length.min())
plt.barh(amplitude,cycles, height = range_only.index.get_level_values('range').length.min())
